In [1]:
import pandas as pd 
dataset1_path = "../data/20240723/window1/dataset1.xlsx"
dataset2_path = "../data/20240723/window1/dataset2.json"

In [2]:
# Load the Excel file from the "DATA" sheet
df_excel = pd.read_excel(dataset1_path, sheet_name='DATA')

# Display the first few rows of the dataframe to understand its structure
print(df_excel.head())
df_excel.info()

      ORDER_ID ORDER_TIME  (PST)  CITY_DISTRICT_ID  RPTG_AMT CURRENCY_CD  \
0  G0000001198             50127                 1   1680.53         RMB   
1  G0000001469             50127                 2   8760.18         RMB   
2  G0000000001             50128                 3   7875.22         RMB   
3  G0000001999             50132                 4  10353.10         RMB   
4  G0000000002             50132                 5  11946.02         RMB   

   ORDER_QTY  
0        1.0  
1        1.0  
2        1.0  
3        1.0  
4        1.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159750 entries, 0 to 159749
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ORDER_ID           159750 non-null  object 
 1   ORDER_TIME  (PST)  159750 non-null  object 
 2   CITY_DISTRICT_ID   159750 non-null  int64  
 3   RPTG_AMT           159750 non-null  float64
 4   CURRENCY_CD        159750 non-null  object 


In [3]:
# Check for unique ORDER_IDs
duplicate_order_ids = df_excel[df_excel.duplicated('ORDER_ID', keep=False)]

# Print the rows with conflicting ORDER_IDs
duplicate_order_ids

,ORDER_ID,ORDER_TIME (PST),CITY_DISTRICT_ID,RPTG_AMT,CURRENCY_CD,ORDER_QTY


In [4]:
# Read JSON file as DataFrame
df_json = pd.read_json(dataset2_path)

# Display the first few rows of the dataframe to understand its structure
print(df_json.head())
df_json.info()

      ORDER_ID  ORDER_TIME_PST SHIP_TO_DISTRICT_NAME SHIP_TO_CITY_CD  \
0  G0000159216          110153                   河东区              天津   
1  G0000159982          110153                   集美区              厦门   
2  G0000163349          110153                   茂南区              茂名   
3  G0000164476          110153                   槐荫区              济南   
4  G0000166038          110153                   海城区              北海   

   RPTG_AMT CURRENCY_CD  ORDER_QTY  
0   7875.22         RMB          1  
1  17520.35         RMB          2  
2  10353.10         RMB          1  
3   8760.18         RMB          1  
4   7078.76         RMB          1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40236 entries, 0 to 40235
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ORDER_ID               40236 non-null  object 
 1   ORDER_TIME_PST         40236 non-null  int64  
 2   SHIP_TO_DISTRICT_NAME  4

In [8]:
# Check for unique ORDER_IDs
duplicate_order_ids = df_json[df_json.duplicated('ORDER_ID', keep=False)]
uniq_cnt_order_id_json = df_json["ORDER_ID"].unique().shape[0]
# Print the rows with conflicting ORDER_IDs
duplicate_order_ids

,ORDER_ID,ORDER_TIME_PST,SHIP_TO_DISTRICT_NAME,SHIP_TO_CITY_CD,RPTG_AMT,CURRENCY_CD,ORDER_QTY


In [9]:
uniq_cnt_order_id_json = df_json["ORDER_ID"].unique().shape[0]
print(f"Unique count of order id in json df:{uniq_cnt_order_id_json}")
uniq_cnt_order_id_excel = df_excel["ORDER_ID"].unique().shape[0]
print(f"Unique count of order id in json df:{uniq_cnt_order_id_excel}")

Unique count of order id in json df:40236
Unique count of order id in json df:159750


In [10]:
# Perform an inner join on ORDER_ID
df_joined = pd.merge(df_excel, df_json, on='ORDER_ID', how='inner')
df_joined

# Dataset 1 and 2 cannot be joined, they are likely the same datasets just in different source formats. 

,ORDER_ID,ORDER_TIME (PST),CITY_DISTRICT_ID,RPTG_AMT_x,CURRENCY_CD_x,ORDER_QTY_x,ORDER_TIME_PST,SHIP_TO_DISTRICT_NAME,SHIP_TO_CITY_CD,RPTG_AMT_y,CURRENCY_CD_y,ORDER_QTY_y


# Data Quality Checks

In [11]:
# Define a function to check if ORDER_QTY fullfills constraints, for INT types
def is_not_integer(value):
    try:
        int_value = int(value)
        if int_value > 0:
            return int_value != value
    except (ValueError, TypeError):
        return True

# Apply the function to the ORDER_QTY column to filter non-integer values
non_integer_count = df_json['ORDER_QTY'].apply(is_not_integer).sum()
non_integer_count_xls = df_excel['ORDER_QTY'].apply(is_not_integer).sum()

# Print the count of non-integer ORDER_QTY
print(f"Count of non-integer ORDER_QTY in df_json: {non_integer_count}")
print(f"Count of non-integer ORDER_QTY in df_excel: {non_integer_count_xls}")

Count of non-integer ORDER_QTY in df_json: 0
Count of non-integer ORDER_QTY in df_excel: 38


In [12]:
# Define a function to check if CURRENCY_CD fullfills constraints, for ENUM types

# Get unique values of CURRENCY_CD in df_excel
unique_currency_cd_excel = df_excel['CURRENCY_CD'].unique()
print("Unique values of CURRENCY_CD in df_excel:")
print(unique_currency_cd_excel)

# Get unique values of CURRENCY_CD in df_json
unique_currency_cd_json = df_json['CURRENCY_CD'].unique()
print("\nUnique values of CURRENCY_CD in df_json:")
print(unique_currency_cd_json)

# Count rows in df_excel where CURRENCY_CD is not 'RMB'
count_not_rmb_excel = df_excel[df_excel['CURRENCY_CD'] != 'RMB'].shape[0]
print(f"\nCount of rows in df_excel where CURRENCY_CD is not 'RMB': {count_not_rmb_excel}")

# Count rows in df_json where CURRENCY_CD is not 'RMB'
count_not_rmb_json = df_json[df_json['CURRENCY_CD'] != 'RMB'].shape[0]
print(f"Count of rows in df_json where CURRENCY_CD is not 'RMB': {count_not_rmb_json}")

Unique values of CURRENCY_CD in df_excel:
['RMB' 'USD']

Unique values of CURRENCY_CD in df_json:
['RMB']

Count of rows in df_excel where CURRENCY_CD is not 'RMB': 37
Count of rows in df_json where CURRENCY_CD is not 'RMB': 0


In [18]:
# Type checks for dollar amounts using floats with 2 decimals. For RPTG_AMT

# Function to check if values are floats with exactly 2 decimal places
def check_two_decimals(df, column):
    # Check if the values are numeric
    is_numeric = pd.to_numeric(df[column], errors='coerce').notnull()
    
    # Check if the values have exactly 2 decimal places
    decimal_check = df[is_numeric][column].apply(lambda x: round(x, 2) == x)
    
    # Check if the values are greater than 0
    greater_than_zero = df[decimal_check][column].apply(lambda x: x > 0)
    
    # Combine both checks
    valid_check = is_numeric & decimal_check & greater_than_zero
    
    # Count invalid values
    invalid_count = (~valid_check).sum()
    
    return invalid_count, valid_check

# Check RPTG_AMT in df_excel
invalid_count_excel, valid_check_excel = check_two_decimals(df_excel, 'RPTG_AMT')
print(f"Count of rows in df_excel where RPTG_AMT is not a float with 2 decimal places: {invalid_count_excel}")

# Check RPTG_AMT in df_json
invalid_count_json, valid_check_json = check_two_decimals(df_json, 'RPTG_AMT')
print(f"Count of rows in df_json where RPTG_AMT is not a float with 2 decimal places: {invalid_count_json}")

Count of rows in df_excel where RPTG_AMT is not a float with 2 decimal places: 0
Count of rows in df_json where RPTG_AMT is not a float with 2 decimal places: 0


In [14]:
# Type checks for CITY_DISTRICT_ID of type ENUM, check if values are within known mapping tables 

# Load the Excel file from the "CITY_DISTRICT_MAP" sheet
df_city_district_map = pd.read_excel(dataset1_path, sheet_name='CITY_DISTRICT_MAP')

# Display the first few rows of the dataframe to understand its structure
print(df_city_district_map.head())

# Print all unique values of SHIP_TO_CITY_CD
unique_city_cd = df_city_district_map['SHIP_TO_CITY_CD'].unique()
unique_city_cd_cnt = unique_city_cd.shape[0]
print("\nUnique values of SHIP_TO_CITY_CD in df_city_district_map:")
print(unique_city_cd)
print("\nCount of Unique values of SHIP_TO_CITY_CD in df_city_district_map:")
print(unique_city_cd_cnt)

# Print all unique values of SHIP_TO_DISTRICT_NAME
unique_district_name = df_city_district_map['SHIP_TO_DISTRICT_NAME'].unique()
unique_district_name_cnt = unique_district_name.shape[0]
print("\nUnique values of SHIP_TO_DISTRICT_NAME in df_city_district_map:")
print(unique_district_name)
print("\nCount of Unique values of SHIP_TO_DISTRICT_NAME in df_city_district_map:")
print(unique_district_name_cnt)

   CITY_DISTRICT_ID SHIP_TO_CITY_CD SHIP_TO_DISTRICT_NAME
0                 1              厦门                   集美区
1                 2              宣城                   宣州区
2                 3              沈阳                   和平区
3                 4              合肥                   瑶海区
4                 5              扬州                   广陵区

Unique values of SHIP_TO_CITY_CD in df_city_district_map:
['厦门' '宣城' '沈阳' '合肥' '扬州' '吉安' '昆明' '上海' '佛山' '北京' '漳州' '成都' '重庆' '蚌埠'
 '广州' '镇江' '钦州' '唐山' '营口' '苏州' '海口' '湛江' '徐州' '九江' '平顶山' '铜仁' '长沙' '常州'
 '眉山' '温州' '济南' '哈尔滨' '贵阳' '西安' '衢州' '贺州' '东营' '玉林' '秦皇岛' '武汉' '长春' '无锡'
 '杭州' '连云港' '菏泽' '松原' '金华' '大同' '南通' '石家庄' '襄阳' '新乡' '晋中' '郑州' '洛阳' '丹东'
 '台州' '青岛' '曲靖' '红河哈尼族彝族自治州' '深圳' '淮北' '嘉兴' '乌鲁木齐' '天津' '双鸭山' '南京' '惠州'
 '佳木斯' '泸州' '阜阳' '保定' '泉州' '南充' '泰州' '汕头' '锦州' '淄博' '南昌' '廊坊' '鹤岗' '咸宁'
 '宁波' '凉山彝族自治州' '福州' '吉林' '宜昌' '赣州' '池州' '绵阳' '济宁' '亳州' '西宁' '葫芦岛' '文昌'
 '兴安盟' '东莞' '齐齐哈尔' '黄冈' '荆州' '抚州' '揭阳' '淮南' '芜湖' '黄石' '枣庄' '淮安' '张家口' '临沂'
 '绥化' '南宁' '盐

In [17]:
# Find rows where CITY_DISTRICT_ID in df_excel is not present in df_city_district_map
invalid_city_district_ids = df_excel[~df_excel['CITY_DISTRICT_ID'].isin(df_city_district_map['CITY_DISTRICT_ID'])]
invalid_city_district_ids_cnt  = invalid_city_district_ids.shape[0]

# Display the rows with invalid CITY_DISTRICT_ID
print(invalid_city_district_ids)
print(f"Count of invalid city district_ids in dataset1: {invalid_city_district_ids_cnt}")

           ORDER_ID ORDER_TIME  (PST)  CITY_DISTRICT_ID  RPTG_AMT CURRENCY_CD  \
130260  G0000135715            102931            999999   8760.18         RMB   
130261  G0000133962            102931            999999   7078.76         RMB   
130262  G0000136128            102931            999999   7963.72         RMB   
130263  G0000127580            102931            999999   7875.22         RMB   
130264  G0000131236            102931            999999   7078.76         RMB   
130265  G0000132568            102931            999999   7963.72         RMB   
130266  G0000134560            102931            999999   7875.22         RMB   
130267  G0000133862            102931            999999  10353.10         RMB   
130268  G0000133863            102931            999999   1325.66         RMB   
130269  G0000135936            102932            999999   7078.76         RMB   
130270  G0000129971            102932            999999   8760.18         RMB   
130271  G0000135324         

In [24]:
# Find rows where SHIP_TO_CITY_CD and SHIP_TO_DISTRICT_NAME in df_json is not present in df_city_district_map

# Extract CITY_CD and DISTRICT_NAME from df_json
city_cd_json = df_json['SHIP_TO_CITY_CD'].unique()
district_name_json = df_json["SHIP_TO_DISTRICT_NAME"].unique()

# Extract CITY_CD and DISTRICT_NAME from the mapping table
city_cd_map = df_city_district_map['SHIP_TO_CITY_CD'].unique()
district_name_map = df_city_district_map['SHIP_TO_DISTRICT_NAME'].unique()

# Find CITY_CD and DISTRICT_NAME from JSON that are not in the mapping table
missing_city_cd = set(city_cd_json) - set(city_cd_map)
missing_district_name = set(district_name_json) -  set(district_name_map)

print(missing_city_cd, missing_district_name)
print(len(missing_city_cd), len(missing_district_name))

{'山南', '莱芜'} {'墨竹工卡县', '右玉县', '合阳县', '国营山荣农场', '遂平县', '吉木乃县', '东凤镇', '横州市', '阿克塞哈萨克族自治县', '关岭布依族苗族自治县', '东区街道', '凌云县', '札达县', '射洪县', '泸溪县', '蓬溪县', '伊春区', '特克斯县', '武功县', '乃东区', '喜德县', '贡嘎县', '凤岗镇', '富民县', '志丹县', '罗甸县', '木垒哈萨克自治县', '灵台县', '洪江市', '任泽区', '小榄镇', '邢台县', '大英县', '丹巴县', '德保县', '祁阳市', '河南蒙古族自治县', '巴马瑶族自治县', '常平镇', '平塘县', '呼中区', '乌拉特后旗', '其它区', '永年县', '米林县', '林甸县', '交口县', '甘泉县', '太谷县', '库车县', '松潘县', '白河县', '丰镇市', '库车市', '连山壮族瑶族自治县', '繁昌区', '尼勒克县', '茶山镇', '茌平县', '柏乡县', '石拐区', '松山湖管委会', '石渠县', '寮步镇', '洞口县', '兴仁县', '莞城街道', '两当县', '塘厦镇', '泰宁县', '平乐县', '漾濞彝族自治县', '延川县', '类乌齐县', '天祝藏族自治县', '涞源县', '洛川县', '黔西市', '西吉县', '碌曲县', '沅陵县', '东兰县', '峡江县', '额尔古纳市', '永胜县', '沧源佤族自治县'}
2 86


In [26]:
# Filter df_json rows which cannot be joined to mapping table
unjoinable_rows = df_json[~df_json['SHIP_TO_CITY_CD'].isin(set(city_cd_map)) | ~df_json['SHIP_TO_DISTRICT_NAME'].isin(set(district_name_map))]

unjoinable_rows

,ORDER_ID,ORDER_TIME_PST,SHIP_TO_DISTRICT_NAME,SHIP_TO_CITY_CD,RPTG_AMT,CURRENCY_CD,ORDER_QTY
88,G0000166194,110159,松潘县,阿坝藏族羌族自治州,7875.22,RMB,1
557,G0000167260,110234,伊春区,伊春,8760.18,RMB,1
719,G0000162868,110247,延川县,延安,8760.18,RMB,1
789,G0000163360,110252,国营山荣农场,乐东黎族自治县,7875.22,RMB,1
1014,G0000145199,110310,凌云县,百色,7875.22,RMB,1
...,...,...,...,...,...,...,...
39113,G0000204735,114735,祁阳市,永州,8760.18,RMB,1
39523,G0000205879,114748,泸溪县,湘西土家族苗族自治州,8760.18,RMB,1
39598,G0000190816,114751,钢城区,莱芜,6105.31,RMB,1
39886,G0000199129,114800,涞源县,保定,10353.10,RMB,1


In [27]:
# Determine the reason why the rows are unjoinable

# Find rows where DISTRICT is not in the mapping table
unjoinable_due_to_district = unjoinable_rows[~unjoinable_rows['SHIP_TO_DISTRICT_NAME'].isin(set(district_name_map))]

# Find rows where CITY is not in the mapping table
unjoinable_due_to_city = unjoinable_rows[~unjoinable_rows['SHIP_TO_CITY_CD'].isin(set(city_cd_map))]

# Find rows where both CITY and DISTRICT pair is not in the mapping table
unjoinable_due_to_both = unjoinable_rows[
    (~unjoinable_rows['SHIP_TO_DISTRICT_NAME'].isin(set(district_name_map))) &
    (~unjoinable_rows['SHIP_TO_CITY_CD'].isin(set(city_cd_map)))
]

# Get counts for each category
count_due_to_district = unjoinable_due_to_district.shape[0]
count_due_to_city = unjoinable_due_to_city.shape[0]
count_due_to_both = unjoinable_due_to_both.shape[0]

count_due_to_district, count_due_to_city, count_due_to_both


(121, 3, 2)

# Translation Feature - Wikipedia Scraping Solution

In [ ]:

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_cities_in_China"

# Use pandas to read the HTML tables from the page
tables = pd.read_html(url)

# Iterate through tables to find the one with the list of cities
for i, table in enumerate(tables):
    if 'Province' in table.columns and 'City' in table.columns:
        df_cities = table
        break

# Display the first few rows of the dataframe to verify
print(df_cities.head())
print(df_cities.shape[0])

In [ ]:
    # Create a function to check if a value is a substring of any entry in a column
    def is_substring(value, series):
        return any(value in str(city) for city in series)

# Remove duplicates from df_json["SHIP_TO_CITY_CD"]
df_json_unique = df_json.drop_duplicates(subset=['SHIP_TO_CITY_CD']).copy()

# Apply the function to each unique city code in df_json_unique
df_json_unique.loc[:, 'city_match'] = df_json_unique['SHIP_TO_CITY_CD'].apply(lambda x: is_substring(x, df_cities['Chinese']))

# Filter rows where city_match is False
df_json_no_match = df_json_unique[df_json_unique['city_match'] == False]
# Remove duplicates from df_city_district_map["SHIP_TO_CITY_CD"]
df_city_district_map_unique = df_city_district_map.drop_duplicates(subset=['SHIP_TO_CITY_CD']).copy()

# Apply the function to each unique city code in df_city_district_map_unique
df_city_district_map_unique.loc[:, 'city_match'] = df_city_district_map_unique['SHIP_TO_CITY_CD'].apply(lambda x: is_substring(x, df_cities['Chinese']))

# Count the number of rows where city_match is True
count_matches = df_json_unique['city_match'].sum()
count_matches_excel = df_city_district_map_unique['city_match'].sum()

# Print the counts
print(count_matches)
print(df_json_unique["SHIP_TO_CITY_CD"].unique().shape[0])

print(count_matches_excel)
print(df_city_district_map_unique["SHIP_TO_CITY_CD"].unique().shape[0])
print(df_json_no_match)